In [28]:
import os  
from dotenv import load_dotenv  
import json  
from typing import Annotated, Literal  
from pydantic import ValidationError  
from azure.identity import DefaultAzureCredential, get_bearer_token_provider  
from langchain.agents import AgentExecutor, create_openai_tools_agent  
from langchain.chains import LLMChain  
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder  
from langchain.tools import tool  
from langchain_core.messages import ToolMessage  
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.pydantic_v1 import BaseModel, Field  
from langchain_openai import ChatOpenAI  
from langchain_openai.chat_models.azure import AzureChatOpenAI  
from langgraph.checkpoint.sqlite import SqliteSaver  
from langgraph.graph import END, MessageGraph, StateGraph  
from langgraph.graph.message import add_messages  
from langgraph.prebuilt.tool_node import ToolNode  
import os  
from dotenv import load_dotenv  
from langchain_openai.chat_models.azure import AzureChatOpenAI  
  
# Load environment variables from a .env file located in the same directory as this script.  
# The .env file contains key-value pairs of environment variables, which helps in managing configuration outside the codebase.  
load_dotenv()  
  
# Retrieve environment variables using os.getenv which reads the environment variables from the system.  
# OPENAI_API_VERSION specifies the API version of the OpenAI service being used.  
api_version = os.getenv("OPENAI_API_VERSION")  
  
# ENDPOINT_URL is the URL where the OpenAI services are hosted. This is required to make API calls to OpenAI.  
endpoint = os.getenv("ENDPOINT_URL")  
  
# DEPLOYMENT_NAME is a specific configuration name for the deployment on Azure. This helps Azure to manage different versions or configurations.  
deployment = os.getenv("DEPLOYMENT_NAME")  
  
# AZURE_OPENAI_ENDPOINT is the endpoint URL for Azure OpenAI specific to your deployment. This URL is used to send requests to Azure's OpenAI service.  
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
  
# AZURE_OPENAI_API_KEY is the secret key required to authenticate requests to Azure OpenAI. This key should be kept secure and not exposed in the code.  
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")  
  
# Initialize the AzureChatOpenAI object with specific parameters:  
# - temperature: Controls the randomness of the response. Setting it to 0 makes the model deterministic, which means the same input will always produce the same output.  
# - deployment_name: Specifies which deployment of the model to use, allowing for version control and experimentation with different model configurations.  
# This object is used to interact with the Azure OpenAI service, sending inputs and receiving generated text responses.  
llm = AzureChatOpenAI(temperature=0, deployment_name=deployment)   

In [29]:
# Using the 'invoke' method to send a request to the AzureChatOpenAI model.  
# The input to the 'invoke' method is a string that represents a prompt to the AI model.  
# In this case, the prompt is 'tell me a joke', which instructs the AI to generate a joke.  
# The AzureChatOpenAI model processes this prompt and returns a response based on the trained AI model.  
# This response is typically a string containing the joke generated by the model.  
response = llm.invoke('tell me a joke')  
  
# The response from the AI model is stored in the variable 'response'.  
# You can now use this 'response' variable in your application to display the joke or log it as needed.  
print(response)  # This line prints the joke to the console.  

content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 11, 'total_tokens': 27, 'completion_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_0165350fbb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-08aa3eac-a7be-4166-ba2c-8d67e52c6a16-0' usage_metadata={'inpu

### Output Breakdown  
  
### Main Content  
- **Text**: "Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"  
  
### Additional Arguments  
- **additional_kwargs**: {}  
  
### Response Metadata  
- **token_usage**:  
  - **completion_tokens**: 16  
  - **prompt_tokens**: 11  
  - **total_tokens**: 27  
  - **completion_tokens_details**: None  
- **model_name**: "gpt-3.5-turbo-0125"  
- **system_fingerprint**: "fp_0165350fbb"  

### ID and Usage Metadata  
- **id**: "run-a73be9c1-fe1a-426b-b629-213c9c6eb053-0"  
- **usage_metadata**:  
  - **input_tokens**: 11  
  - **output_tokens**: 16  
  - **total_tokens**: 27  

In [30]:
from pydantic import BaseModel, Field
from typing import Optional

class Classification(BaseModel):
    is_energy_related: bool = Field(
        ...,
        description="Indicates if the text is related to energy"
    )
    relevance_score: Optional[int] = Field(
        None,
        description="Degree of relevance to energy, with 10 being very highly related",
        ge=1, le=10
    )
    # energy_category: Optional[str] = Field(
    #     None,
    #     description="The specific category of the energy-related text",
    #     enum=[
    #         "wells", "pipelines", "infrastructure", "imagery", "weather",
    #         "environmental", "geology", "seismic", "geomatics", "renewables",
    #         "emissions", "basemaps", "bathymetry"
    #     ]
    # )


# example_classification = Classification(
#     is_energy_related=True,
#     relevance_score=8,
#     energy_category="renewables"
# )
# print(example_classification)

from langchain.prompts import ChatPromptTemplate

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


# Create an instance of Azure OpenAI
llm = AzureChatOpenAI(temperature = 0, deployment_name="gpt-35-turbo-0125").with_structured_output(
    Classification
)


chain = tagging_prompt | llm

In [31]:
inp = "The new geothermal plant will significantly enhance the renewable energy capacity of the region. It involves drilling new wells and setting up infrastructure to harness geothermal energy efficiently."
chain.invoke({"input": inp})

Classification(is_energy_related=True, relevance_score=None)

In [32]:
from pydantic import BaseModel, Field  
from typing import Optional  
from langchain.prompts import ChatPromptTemplate  
  
# Define a Pydantic model for classification data related to the energy sector.  
class Classification(BaseModel):  
    # Field to indicate if the text is associated with the energy sector.  
    is_energy_related: bool = Field(  
        ...,  # Ellipsis used to mark the field as required.  
        description="Indicates whether the text is associated with specific aspects of the energy sector, such as wells, pipelines, renewables, subsurface, emissions, geology, land ownership or infrastructure, environmental impact, or seismic activities."  
    )  
    # Optional field for scoring the relevance of the text to the energy sector.  
    relevance_score: Optional[int] = Field(  
        None,  # Default value is None, making this field optional.  
        description="Evaluates the text's relevance to key energy categories, with a score of 10 representing strong alignment.",  
        ge=1,  # Minimum value is 1.  
        le=10  # Maximum value is 10.  
    )  
    # Optional field to categorize the energy-related aspect of the text.  
    energy_category: Optional[str] = Field(  
        None,  # Default value is None, making this field optional.  
        description="The specific category of the energy-related text",  
        enum=[  # List of allowed values for the category.  
            "wells", "pipelines", "infrastructure", "imagery", "weather",  
            "environmental", "geology", "seismic", "geomatics", "renewables",  
            "emissions", "basemaps", "bathymetry"  
        ]  
    )  
  
# Create a prompt template for extracting classification information from a passage.  
tagging_prompt = ChatPromptTemplate.from_template(  
    """Extract the desired information from the following passage.  
    Only extract the properties mentioned in the 'Classification' function.  
    Passage: {input}"""  
)  
  
# Create an instance of Azure OpenAI with structured output based on the Classification model.  
llm = AzureChatOpenAI(  
    temperature=0,  # Set the temperature to 0 for deterministic results.  
    deployment_name="gpt-35-turbo-0125"  # Specify the deployment name.  
).with_structured_output(Classification)  
  
# Chain the prompt template with the language model to process input.  
chain = tagging_prompt | llm  

In [33]:
# Define the input text which needs to be analyzed for its relevance to energy topics.  
inp = "The new geothermal plant will significantly enhance the renewable energy capacity of the region. It involves drilling new wells and setting up infrastructure to harness geothermal energy efficiently."  
  
# Use the `chain` to invoke the language model with the input text.  
# This will analyze the text based on the `Classification` model and the rules defined in the `tagging_prompt`.  
result = chain.invoke({"input": inp})  
  
# `result` now contains the structured output from the model, analyzing aspects like whether the text is energy-related,  
# its relevance score, and the specific energy category, based on the input provided.  
print(result)

is_energy_related=True relevance_score=None energy_category=None


In [34]:
import pandas as pd  
  
# Define the schema for the data to ensure each column is read with the correct data type.  
data_schema = {  
    'title': 'object',  # 'title' as a generic object, typically string.  
    'url': 'object',  # 'url' as a generic object.  
    'type': 'object',  # 'type' as a generic object.  
    'tags': 'object',  # 'tags' as a generic object.  
    'description': 'object',  # 'description' as a generic object.  
    'thumbnailUrl': 'object'  # 'thumbnailUrl' as a generic object.  
}  
  
# Define the file path to the CSV file.  
file_path = r'C:\Users\hqxb\Downloads\Output\output\export-2.csv'  
  
# Read the CSV file using the defined schema to ensure correct data type assignment.  
export_data_frame  = pd.read_csv(file_path, dtype=data_schema)  
  
# Initialize additional columns in the DataFrame to store analysis results:  
export_data_frame ['relevance_score'] = None  # Placeholder for relevance scores, to be filled after analysis.  
export_data_frame ['energy_category'] = None  # Placeholder for energy category, to be determined later.  
export_data_frame ['is_valid_url'] = False  # Placeholder for URL validity, to be checked later.  
  
  
# `export_data_frame` is now prepared with additional columns and ready for data processing or analysis.  

In [35]:
export_data_frame.head()

,title,url,type,tags,description,thumbnailUrl,relevance_score,energy_category,is_valid_url
0,LTDL_Tool/NLCD_2016_Land_Cover_L48,https://srfs.wr.usgs.gov/arcgis/rest/services/...,Map Service,"USGS - Land Treatments Digital Library,EnergyA...","The U.S. Geological Survey (USGS), in partners...",https://srfs.wr.usgs.gov/arcgis/rest/services/...,None,None,False
1,BioGas/CNG_EV,https://www.usda.gov/giseas1/rest/services/Bio...,Map Service,"USDA - Enterprise Application Services,EnergyA...",Natural Gas Fueling Stations: This layer consi...,https://www.usda.gov/giseas1/rest/services/Bio...,None,None,False
2,EnergyInvestments/EnergyInvestments_EnergyEffi...,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False
3,EnergyInvestments/EnergyInvestments_Anaerobic,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False
4,EnergyInvestments/EnergyInvestments_Hydroelectric,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False


In [36]:
import pandas as pd  
import requests  
  
# Function to check if a URL is valid  
def check_url(url):  
    try:  
        # Make a HEAD request to the URL to check its validity.  
        response = requests.head(url, allow_redirects=True, timeout=5)  
        # Return True if the status code is 200, indicating the URL is valid.  
        return response.status_code == 200  
    except requests.RequestException:  
        # Return False if any exception occurs during the request.  
        return False  
  
# Function to classify the title based on energy relevance and validate URLs  
def classify_and_validate(data_frame, limit_rows=None):  
    # If limit_rows is specified, limit the DataFrame to the first 'limit_rows' rows.  
    if limit_rows:  
        data_frame = data_frame.head(limit_rows)  
      
    # Iterate through each row in the DataFrame  
    for index, row in data_frame.iterrows():  
        title = row['title']  
          
        # Run the classification on the title  
        classification_result = chain.invoke({"input": title})  
          
        # Unpack the classification result and update the DataFrame  
        data_frame.at[index, 'is_energy_related'] = classification_result.is_energy_related  
        data_frame.at[index, 'relevance_score'] = classification_result.relevance_score  
        data_frame.at[index, 'energy_category'] = classification_result.energy_category  
          
        # Validate the URL and update the DataFrame  
        data_frame.at[index, 'is_valid_url'] = check_url(row['url'])  
      
    # Return the updated DataFrame  
    return data_frame  
  
# Apply classification and URL validation to the first 25 rows of the DataFrame  
export_data_frame_limited = classify_and_validate(export_data_frame, limit_rows=5)  
  


C:\Users\hqxb\AppData\Local\Temp\ipykernel_16120\1150711754.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame.at[index, 'is_energy_related'] = classification_result.is_energy_related
C:\Users\hqxb\AppData\Local\Temp\ipykernel_16120\1150711754.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_frame.at[index, 'is_energy_related'] = classification_result.is_energy_related


In [37]:
# Display the head of the processed DataFrame to inspect the first few rows  
export_data_frame_limited.head()

,title,url,type,tags,description,thumbnailUrl,relevance_score,energy_category,is_valid_url,is_energy_related
0,LTDL_Tool/NLCD_2016_Land_Cover_L48,https://srfs.wr.usgs.gov/arcgis/rest/services/...,Map Service,"USGS - Land Treatments Digital Library,EnergyA...","The U.S. Geological Survey (USGS), in partners...",https://srfs.wr.usgs.gov/arcgis/rest/services/...,None,None,False,False
1,BioGas/CNG_EV,https://www.usda.gov/giseas1/rest/services/Bio...,Map Service,"USDA - Enterprise Application Services,EnergyA...",Natural Gas Fueling Stations: This layer consi...,https://www.usda.gov/giseas1/rest/services/Bio...,None,None,False,True
2,EnergyInvestments/EnergyInvestments_EnergyEffi...,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False,True
3,EnergyInvestments/EnergyInvestments_Anaerobic,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False,True
4,EnergyInvestments/EnergyInvestments_Hydroelectric,https://www.usda.gov/giseas1/rest/services/Ene...,Map Service,"USDA - Enterprise Application Services,EnergyA...",The USDA Energy Investments contains informati...,https://www.usda.gov/giseas1/rest/services/Ene...,None,None,False,True


## Enhancing GIS Workflows with Large Language Models (LLMs)  
  
Integrating Large Language Models (LLMs) into the preprocessing of potential layers and managing the metadata of existing layers in an Enterprise GIS system can streamline data integration processes and improve the organization and updating of GIS data.  
  
### Workflow for Preprocessing and Metadata Management  
  
#### 1. Metadata Extraction and Analysis  
- **Automate Metadata Reading**: Utilize LLMs to automatically extract and interpret metadata, enhancing understanding and usability.  
- **Semantic Analysis**: Employ semantic analysis to evaluate the context and relevance, identifying outdated or redundant layers.  
  
#### 2. Data Classification and Tagging  
- **Enhance Metadata**: Use LLMs to enrich metadata with more descriptive and accurate tags.  
- **Classify Data**: Automatically categorize GIS layers to streamline management and accessibility.  
  
#### 3. Validation and Quality Checks  
- **URL Validation**: Ensure all external references in metadata are up-to-date and accessible.  
- **Data Consistency**: Maintain consistency and accuracy in data and metadata, applying corrections as needed.  
  
#### 4. Layer Optimization  
- **Layer Consolidation**: Reduce redundancy by consolidating similar or duplicate layers.  
- **Update Recommendations**: Leverage insights from LLMs to guide updates or archival of layers based on their utility and relevance.  
  
### Additional Implementations of LLMs in GIS Enterprise Systems  
  
1. **Automated Documentation**: Generate detailed, automatic documentation for GIS layers and projects.  
2. **Training and Tutorials**: Develop adaptive training modules that provide context-aware guidance.  
3. **Predictive Analytics**: Predict trends and suggest new data layers based on analytics, enhancing foresight and planning.  
  
### Conclusion and Considerations  
  
While LLMs can significantly enhance GIS enterprise systems by automating and enriching workflows, it is crucial to remember that they are not a panacea. LLMs should be viewed as powerful tools that extend and elevate existing workflows which already include checks, filters, and other data management practices. By integrating LLMs wisely, we can take GIS data management and usability to the next level, fostering a more efficient and intuitive system.  